In [15]:
import os
import pandas as pd
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS
from itertools import product
from IPython.display import display, HTML

In [25]:
_LOG_DIR = "logs/train_kernel"

ITERATIONS = [1, 3]

def get_df(domain):
  d = {
    "config": [],
    "mse": [],
    "f1": [],
    "nonzero_weights": [],
  }

  CONFIGS = list(product(MODELS, ITERATIONS))
  for config in CONFIGS:
    model, iterations = config
    log_file = "_".join([model, "llg", "ipc2023-learning-"+domain, "wl", str(iterations)])+".log"
    log_file = _LOG_DIR + "/" + log_file
    
    if not os.path.exists(log_file):
      continue

    stats = {
      "config": "_".join([model, "wl", str(iterations)])
    }

    for line in open(log_file, 'r').readlines():
      toks = line.split()
      if "train_mse" in line:
        stats["mse"] = float(toks[-1])
      elif "train_f1_macro" in line:
        stats["f1"] = float(toks[-1])
      elif "zero_weights" in line:
        weights = int(toks[1].split('/')[1])
        zeros = int(toks[1].split('/')[0])
        stats["nonzero_weights"] = weights - zeros
    
    if len(stats) != len(d):
      continue

    for key in stats:
      d[key].append(stats[key])

  df = pd.DataFrame(d)
  return df

In [26]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain)
  df = get_df(domain)
  display(df)

blocksworld


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.09,0.93,6219
1,linear-svr_wl_1,0.32,0.45,6150
2,linear-svr_wl_3,0.06,0.99,419796
3,ridge_wl_1,0.25,0.69,6218
4,ridge_wl_3,0.03,0.99,431584
5,lasso_wl_1,3.74,0.23,6
6,lasso_wl_3,3.74,0.23,7


childsnack


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,1.39,0.72,556
1,linear-regression_wl_3,7.72,0.41,16006
2,linear-svr_wl_1,1.71,0.81,552
3,linear-svr_wl_3,1.80,0.81,15577
4,ridge_wl_1,1.40,0.72,501
5,ridge_wl_3,1.39,0.72,15870
6,lasso_wl_1,10.76,0.07,8
7,lasso_wl_3,9.96,0.07,12


ferry


...

,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.00,1.0,0
1,linear-regression_wl_3,0.00,1.0,0
2,linear-svr_wl_1,0.00,1.0,119
3,linear-svr_wl_3,0.01,1.0,3910
4,ridge_wl_1,0.00,1.0,0
5,ridge_wl_3,0.00,1.0,0
6,lasso_wl_1,0.00,1.0,0
7,lasso_wl_3,0.00,1.0,0


spanner


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,1.65,0.16,229
1,linear-regression_wl_3,14.59,0.09,6312
2,linear-svr_wl_1,1.74,0.20,239
3,linear-svr_wl_3,1.19,0.33,6130
4,ridge_wl_1,1.65,0.15,203
5,ridge_wl_3,1.06,0.28,6224
6,lasso_wl_1,2.14,0.09,4
7,lasso_wl_3,2.14,0.09,4


transport


,config,mse,f1,nonzero_weights
0,linear-regression_wl_1,0.04,0.44,273
1,linear-regression_wl_3,0.00,1.00,7708
2,linear-svr_wl_1,0.05,0.50,257
3,linear-svr_wl_3,0.01,1.00,4916
4,ridge_wl_1,0.04,0.44,218
5,ridge_wl_3,0.00,1.00,7646
6,lasso_wl_1,0.07,0.33,0
7,lasso_wl_3,0.07,0.33,0
